# **Synthetic Data Generation for {process_description, db_spec} JSON Examples**
This notebook generates synthetic datasets for use cases where a natural language process description is converted into a structured database schema and SQL query generation plan.

# 1. Install Dependencies

In [1]:
!pip install groq jsonschema datasets huggingface_hub

  Attempting uninstall: tqdm
    Found existing installation: tqdm 4.66.2
    Uninstalling tqdm-4.66.2:
      Successfully uninstalled tqdm-4.66.2
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
tensorflow 2.19.0 requires numpy<2.2.0,>=1.26.0, but you have numpy 2.2.5 which is incompatible.

[notice] A new release of pip is available: 24.2 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# 2. Authentication & Setup

In [ ]:
import os
os.environ['GROQ_API_KEY'] = 'your_api_key_here'

!huggingface-cli login

# 3. Load Seed Templates

We start with ~10 hand-crafted process_description → db_spec pairs.

In [10]:
import json
import pandas as pd
from typing import List, Dict, Any

# High-Quality Seed Dataset for Database Schema & Query Generation
SEED_EXAMPLES = [
    {
        "id": "ecommerce_platform",
        "business_requirements": "Multi-tenant e-commerce platform supporting multiple vendors, product catalogs, order processing, inventory management, and customer reviews. Need to handle 10M+ products, 1M+ daily orders, with real-time inventory updates.",
        "domain_context": "B2B marketplace with complex pricing tiers, bulk discounts, and international shipping. High-write workload with analytics requirements.",
        "schema_design": {
            "tables": [
                {
                    "name": "tenants",
                    "columns": [
                        {"name": "id", "type": "BIGINT", "constraints": ["PRIMARY KEY", "AUTO_INCREMENT"]},
                        {"name": "name", "type": "VARCHAR(255)", "constraints": ["NOT NULL"]},
                        {"name": "subdomain", "type": "VARCHAR(100)", "constraints": ["UNIQUE", "NOT NULL"]},
                        {"name": "created_at", "type": "TIMESTAMP", "constraints": ["DEFAULT CURRENT_TIMESTAMP"]},
                        {"name": "status", "type": "ENUM('active', 'suspended', 'deleted')", "constraints": ["DEFAULT 'active'"]}
                    ],
                    "indexes": [
                        {"name": "idx_tenants_subdomain", "columns": ["subdomain"], "type": "UNIQUE"},
                        {"name": "idx_tenants_status", "columns": ["status"]}
                    ]
                },
                {
                    "name": "users",
                    "columns": [
                        {"name": "id", "type": "BIGINT", "constraints": ["PRIMARY KEY", "AUTO_INCREMENT"]},
                        {"name": "tenant_id", "type": "BIGINT", "constraints": ["NOT NULL"]},
                        {"name": "email", "type": "VARCHAR(255)", "constraints": ["NOT NULL"]},
                        {"name": "phone", "type": "VARCHAR(20)", "constraints": []},
                        {"name": "first_name", "type": "VARCHAR(100)", "constraints": ["NOT NULL"]},
                        {"name": "last_name", "type": "VARCHAR(100)", "constraints": ["NOT NULL"]},
                        {"name": "user_type", "type": "ENUM('customer', 'vendor', 'admin')", "constraints": ["NOT NULL"]},
                        {"name": "created_at", "type": "TIMESTAMP", "constraints": ["DEFAULT CURRENT_TIMESTAMP"]},
                        {"name": "last_login_at", "type": "TIMESTAMP", "constraints": []},
                        {"name": "is_verified", "type": "BOOLEAN", "constraints": ["DEFAULT FALSE"]}
                    ],
                    "indexes": [
                        {"name": "idx_users_tenant_email", "columns": ["tenant_id", "email"], "type": "UNIQUE"},
                        {"name": "idx_users_type", "columns": ["user_type"]},
                        {"name": "idx_users_created_at", "columns": ["created_at"]},
                        {"name": "idx_users_last_login", "columns": ["last_login_at"]}
                    ]
                },
                {
                    "name": "products",
                    "columns": [
                        {"name": "id", "type": "BIGINT", "constraints": ["PRIMARY KEY", "AUTO_INCREMENT"]},
                        {"name": "tenant_id", "type": "BIGINT", "constraints": ["NOT NULL"]},
                        {"name": "vendor_id", "type": "BIGINT", "constraints": ["NOT NULL"]},
                        {"name": "sku", "type": "VARCHAR(100)", "constraints": ["NOT NULL"]},
                        {"name": "name", "type": "VARCHAR(500)", "constraints": ["NOT NULL"]},
                        {"name": "description", "type": "TEXT", "constraints": []},
                        {"name": "category_id", "type": "BIGINT", "constraints": ["NOT NULL"]},
                        {"name": "base_price", "type": "DECIMAL(10,2)", "constraints": ["NOT NULL"]},
                        {"name": "cost_price", "type": "DECIMAL(10,2)", "constraints": []},
                        {"name": "weight", "type": "DECIMAL(8,3)", "constraints": []},
                        {"name": "status", "type": "ENUM('draft', 'active', 'discontinued')", "constraints": ["DEFAULT 'draft'"]},
                        {"name": "created_at", "type": "TIMESTAMP", "constraints": ["DEFAULT CURRENT_TIMESTAMP"]},
                        {"name": "updated_at", "type": "TIMESTAMP", "constraints": ["DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP"]}
                    ],
                    "indexes": [
                        {"name": "idx_products_tenant_sku", "columns": ["tenant_id", "sku"], "type": "UNIQUE"},
                        {"name": "idx_products_vendor", "columns": ["vendor_id"]},
                        {"name": "idx_products_category", "columns": ["category_id"]},
                        {"name": "idx_products_status", "columns": ["status"]},
                        {"name": "idx_products_price_range", "columns": ["base_price"]},
                        {"name": "idx_products_search", "columns": ["name", "description"], "type": "FULLTEXT"}
                    ]
                },
                {
                    "name": "inventory",
                    "columns": [
                        {"name": "id", "type": "BIGINT", "constraints": ["PRIMARY KEY", "AUTO_INCREMENT"]},
                        {"name": "product_id", "type": "BIGINT", "constraints": ["NOT NULL"]},
                        {"name": "warehouse_id", "type": "BIGINT", "constraints": ["NOT NULL"]},
                        {"name": "quantity_available", "type": "INT", "constraints": ["DEFAULT 0"]},
                        {"name": "quantity_reserved", "type": "INT", "constraints": ["DEFAULT 0"]},
                        {"name": "reorder_level", "type": "INT", "constraints": ["DEFAULT 10"]},
                        {"name": "max_stock_level", "type": "INT", "constraints": []},
                        {"name": "last_updated", "type": "TIMESTAMP", "constraints": ["DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP"]}
                    ],
                    "indexes": [
                        {"name": "idx_inventory_product_warehouse", "columns": ["product_id", "warehouse_id"], "type": "UNIQUE"},
                        {"name": "idx_inventory_low_stock", "columns": ["quantity_available", "reorder_level"]},
                        {"name": "idx_inventory_warehouse", "columns": ["warehouse_id"]}
                    ]
                },
                {
                    "name": "orders",
                    "columns": [
                        {"name": "id", "type": "BIGINT", "constraints": ["PRIMARY KEY", "AUTO_INCREMENT"]},
                        {"name": "tenant_id", "type": "BIGINT", "constraints": ["NOT NULL"]},
                        {"name": "customer_id", "type": "BIGINT", "constraints": ["NOT NULL"]},
                        {"name": "order_number", "type": "VARCHAR(50)", "constraints": ["UNIQUE", "NOT NULL"]},
                        {"name": "status", "type": "ENUM('pending', 'confirmed', 'processing', 'shipped', 'delivered', 'cancelled')", "constraints": ["DEFAULT 'pending'"]},
                        {"name": "subtotal", "type": "DECIMAL(12,2)", "constraints": ["NOT NULL"]},
                        {"name": "tax_amount", "type": "DECIMAL(12,2)", "constraints": ["DEFAULT 0"]},
                        {"name": "shipping_amount", "type": "DECIMAL(12,2)", "constraints": ["DEFAULT 0"]},
                        {"name": "total_amount", "type": "DECIMAL(12,2)", "constraints": ["NOT NULL"]},
                        {"name": "currency", "type": "CHAR(3)", "constraints": ["DEFAULT 'USD'"]},
                        {"name": "payment_status", "type": "ENUM('pending', 'paid', 'failed', 'refunded')", "constraints": ["DEFAULT 'pending'"]},
                        {"name": "shipping_address_id", "type": "BIGINT", "constraints": []},
                        {"name": "created_at", "type": "TIMESTAMP", "constraints": ["DEFAULT CURRENT_TIMESTAMP"]},
                        {"name": "updated_at", "type": "TIMESTAMP", "constraints": ["DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP"]}
                    ],
                    "indexes": [
                        {"name": "idx_orders_customer", "columns": ["customer_id"]},
                        {"name": "idx_orders_status", "columns": ["status"]},
                        {"name": "idx_orders_payment_status", "columns": ["payment_status"]},
                        {"name": "idx_orders_created_at", "columns": ["created_at"]},
                        {"name": "idx_orders_tenant_date", "columns": ["tenant_id", "created_at"]}
                    ]
                }
            ],
            "relationships": [
                {"from": "users.tenant_id", "to": "tenants.id", "type": "FOREIGN KEY", "on_delete": "CASCADE"},
                {"from": "products.tenant_id", "to": "tenants.id", "type": "FOREIGN KEY", "on_delete": "CASCADE"},
                {"from": "products.vendor_id", "to": "users.id", "type": "FOREIGN KEY", "on_delete": "RESTRICT"},
                {"from": "inventory.product_id", "to": "products.id", "type": "FOREIGN KEY", "on_delete": "CASCADE"},
                {"from": "orders.tenant_id", "to": "tenants.id", "type": "FOREIGN KEY", "on_delete": "CASCADE"},
                {"from": "orders.customer_id", "to": "users.id", "type": "FOREIGN KEY", "on_delete": "RESTRICT"}
            ]
        },
        "performance_considerations": {
            "partitioning": [
                "orders table partitioned by created_at (monthly partitions)",
                "inventory_logs table partitioned by date (daily partitions)"
            ],
            "indexing_strategy": [
                "Composite indexes for multi-tenant queries",
                "Partial indexes for active products only",
                "Covering indexes for common SELECT queries"
            ],
            "caching": [
                "Redis for product catalog with 1-hour TTL",
                "Memcached for user sessions",
                "Application-level caching for category hierarchies"
            ],
            "read_replicas": "3 read replicas for analytics and reporting workloads",
            "connection_pooling": "PgBouncer with 100 connections per pool"
        },
        "queries": {
            "common_operations": [
                {
                    "description": "Get customer's recent orders with product details",
                    "sql": "SELECT o.id, o.order_number, o.status, o.total_amount, o.created_at, GROUP_CONCAT(p.name) as products FROM orders o JOIN order_items oi ON o.id = oi.order_id JOIN products p ON oi.product_id = p.id WHERE o.customer_id = ? AND o.tenant_id = ? ORDER BY o.created_at DESC LIMIT 10",
                    "performance_notes": "Uses composite index on (customer_id, tenant_id, created_at)"
                },
                {
                    "description": "Find low stock products for reordering",
                    "sql": "SELECT p.id, p.name, p.sku, i.quantity_available, i.reorder_level FROM products p JOIN inventory i ON p.id = i.product_id WHERE i.quantity_available <= i.reorder_level AND p.status = 'active' AND p.tenant_id = ?",
                    "performance_notes": "Uses partial index on low stock conditions"
                },
                {
                    "description": "Product search with filters",
                    "sql": "SELECT p.*, AVG(r.rating) as avg_rating FROM products p LEFT JOIN reviews r ON p.id = r.product_id WHERE p.tenant_id = ? AND p.status = 'active' AND (MATCH(p.name, p.description) AGAINST (? IN NATURAL LANGUAGE MODE) OR p.category_id IN (SELECT id FROM categories WHERE name LIKE ?)) AND p.base_price BETWEEN ? AND ? GROUP BY p.id ORDER BY avg_rating DESC, p.created_at DESC LIMIT 20",
                    "performance_notes": "Uses fulltext index for search, category hierarchy index"
                }
            ],
            "analytics_queries": [
                {
                    "description": "Monthly revenue by vendor",
                    "sql": "SELECT u.first_name, u.last_name, SUM(oi.quantity * oi.unit_price) as revenue FROM users u JOIN products p ON u.id = p.vendor_id JOIN order_items oi ON p.id = oi.product_id JOIN orders o ON oi.order_id = o.id WHERE o.status = 'delivered' AND o.created_at >= DATE_SUB(NOW(), INTERVAL 1 MONTH) AND o.tenant_id = ? GROUP BY u.id ORDER BY revenue DESC",
                    "performance_notes": "Benefits from date partitioning on orders table"
                }
            ]
        }
    },

    {
        "id": "social_media_platform",
        "business_requirements": "Social media platform with user profiles, posts, comments, likes, followers, direct messaging, and content moderation. Handle 100M+ users, 1B+ posts, real-time feeds, and content recommendation engine.",
        "domain_context": "High-read, high-write social platform with real-time notifications, content virality patterns, and global distribution requirements.",
        "schema_design": {
            "tables": [
                {
                    "name": "users",
                    "columns": [
                        {"name": "id", "type": "BIGINT", "constraints": ["PRIMARY KEY", "AUTO_INCREMENT"]},
                        {"name": "username", "type": "VARCHAR(50)", "constraints": ["UNIQUE", "NOT NULL"]},
                        {"name": "email", "type": "VARCHAR(255)", "constraints": ["UNIQUE", "NOT NULL"]},
                        {"name": "password_hash", "type": "VARCHAR(255)", "constraints": ["NOT NULL"]},
                        {"name": "display_name", "type": "VARCHAR(100)", "constraints": []},
                        {"name": "bio", "type": "TEXT", "constraints": []},
                        {"name": "avatar_url", "type": "VARCHAR(500)", "constraints": []},
                        {"name": "follower_count", "type": "INT", "constraints": ["DEFAULT 0"]},
                        {"name": "following_count", "type": "INT", "constraints": ["DEFAULT 0"]},
                        {"name": "post_count", "type": "INT", "constraints": ["DEFAULT 0"]},
                        {"name": "is_verified", "type": "BOOLEAN", "constraints": ["DEFAULT FALSE"]},
                        {"name": "is_private", "type": "BOOLEAN", "constraints": ["DEFAULT FALSE"]},
                        {"name": "created_at", "type": "TIMESTAMP", "constraints": ["DEFAULT CURRENT_TIMESTAMP"]},
                        {"name": "last_active_at", "type": "TIMESTAMP", "constraints": []}
                    ],
                    "indexes": [
                        {"name": "idx_users_username", "columns": ["username"], "type": "UNIQUE"},
                        {"name": "idx_users_email", "columns": ["email"], "type": "UNIQUE"},
                        {"name": "idx_users_last_active", "columns": ["last_active_at"]},
                        {"name": "idx_users_follower_count", "columns": ["follower_count"]},
                        {"name": "idx_users_verified", "columns": ["is_verified"]}
                    ]
                },
                {
                    "name": "posts",
                    "columns": [
                        {"name": "id", "type": "BIGINT", "constraints": ["PRIMARY KEY", "AUTO_INCREMENT"]},
                        {"name": "user_id", "type": "BIGINT", "constraints": ["NOT NULL"]},
                        {"name": "content", "type": "TEXT", "constraints": []},
                        {"name": "media_urls", "type": "JSON", "constraints": []},
                        {"name": "post_type", "type": "ENUM('text', 'image', 'video', 'poll')", "constraints": ["DEFAULT 'text'"]},
                        {"name": "like_count", "type": "INT", "constraints": ["DEFAULT 0"]},
                        {"name": "comment_count", "type": "INT", "constraints": ["DEFAULT 0"]},
                        {"name": "share_count", "type": "INT", "constraints": ["DEFAULT 0"]},
                        {"name": "engagement_score", "type": "DECIMAL(10,4)", "constraints": ["DEFAULT 0"]},
                        {"name": "is_promoted", "type": "BOOLEAN", "constraints": ["DEFAULT FALSE"]},
                        {"name": "visibility", "type": "ENUM('public', 'followers', 'private')", "constraints": ["DEFAULT 'public'"]},
                        {"name": "created_at", "type": "TIMESTAMP", "constraints": ["DEFAULT CURRENT_TIMESTAMP"]},
                        {"name": "updated_at", "type": "TIMESTAMP", "constraints": ["DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP"]}
                    ],
                    "indexes": [
                        {"name": "idx_posts_user_created", "columns": ["user_id", "created_at"]},
                        {"name": "idx_posts_engagement", "columns": ["engagement_score", "created_at"]},
                        {"name": "idx_posts_visibility", "columns": ["visibility"]},
                        {"name": "idx_posts_promoted", "columns": ["is_promoted", "created_at"]},
                        {"name": "idx_posts_content", "columns": ["content"], "type": "FULLTEXT"}
                    ]
                },
                {
                    "name": "follows",
                    "columns": [
                        {"name": "id", "type": "BIGINT", "constraints": ["PRIMARY KEY", "AUTO_INCREMENT"]},
                        {"name": "follower_id", "type": "BIGINT", "constraints": ["NOT NULL"]},
                        {"name": "following_id", "type": "BIGINT", "constraints": ["NOT NULL"]},
                        {"name": "status", "type": "ENUM('pending', 'accepted', 'blocked')", "constraints": ["DEFAULT 'accepted'"]},
                        {"name": "created_at", "type": "TIMESTAMP", "constraints": ["DEFAULT CURRENT_TIMESTAMP"]}
                    ],
                    "indexes": [
                        {"name": "idx_follows_unique", "columns": ["follower_id", "following_id"], "type": "UNIQUE"},
                        {"name": "idx_follows_following", "columns": ["following_id", "status"]},
                        {"name": "idx_follows_follower", "columns": ["follower_id", "status"]}
                    ]
                },
                {
                    "name": "likes",
                    "columns": [
                        {"name": "id", "type": "BIGINT", "constraints": ["PRIMARY KEY", "AUTO_INCREMENT"]},
                        {"name": "user_id", "type": "BIGINT", "constraints": ["NOT NULL"]},
                        {"name": "post_id", "type": "BIGINT", "constraints": ["NOT NULL"]},
                        {"name": "created_at", "type": "TIMESTAMP", "constraints": ["DEFAULT CURRENT_TIMESTAMP"]}
                    ],
                    "indexes": [
                        {"name": "idx_likes_unique", "columns": ["user_id", "post_id"], "type": "UNIQUE"},
                        {"name": "idx_likes_post", "columns": ["post_id", "created_at"]},
                        {"name": "idx_likes_user", "columns": ["user_id", "created_at"]}
                    ]
                },
                {
                    "name": "comments",
                    "columns": [
                        {"name": "id", "type": "BIGINT", "constraints": ["PRIMARY KEY", "AUTO_INCREMENT"]},
                        {"name": "post_id", "type": "BIGINT", "constraints": ["NOT NULL"]},
                        {"name": "user_id", "type": "BIGINT", "constraints": ["NOT NULL"]},
                        {"name": "parent_comment_id", "type": "BIGINT", "constraints": []},
                        {"name": "content", "type": "TEXT", "constraints": ["NOT NULL"]},
                        {"name": "like_count", "type": "INT", "constraints": ["DEFAULT 0"]},
                        {"name": "reply_count", "type": "INT", "constraints": ["DEFAULT 0"]},
                        {"name": "created_at", "type": "TIMESTAMP", "constraints": ["DEFAULT CURRENT_TIMESTAMP"]},
                        {"name": "updated_at", "type": "TIMESTAMP", "constraints": ["DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP"]}
                    ],
                    "indexes": [
                        {"name": "idx_comments_post", "columns": ["post_id", "created_at"]},
                        {"name": "idx_comments_user", "columns": ["user_id", "created_at"]},
                        {"name": "idx_comments_parent", "columns": ["parent_comment_id"]},
                        {"name": "idx_comments_thread", "columns": ["post_id", "parent_comment_id", "created_at"]}
                    ]
                }
            ],
            "relationships": [
                {"from": "posts.user_id", "to": "users.id", "type": "FOREIGN KEY", "on_delete": "CASCADE"},
                {"from": "follows.follower_id", "to": "users.id", "type": "FOREIGN KEY", "on_delete": "CASCADE"},
                {"from": "follows.following_id", "to": "users.id", "type": "FOREIGN KEY", "on_delete": "CASCADE"},
                {"from": "likes.user_id", "to": "users.id", "type": "FOREIGN KEY", "on_delete": "CASCADE"},
                {"from": "likes.post_id", "to": "posts.id", "type": "FOREIGN KEY", "on_delete": "CASCADE"},
                {"from": "comments.post_id", "to": "posts.id", "type": "FOREIGN KEY", "on_delete": "CASCADE"},
                {"from": "comments.user_id", "to": "users.id", "type": "FOREIGN KEY", "on_delete": "CASCADE"},
                {"from": "comments.parent_comment_id", "to": "comments.id", "type": "FOREIGN KEY", "on_delete": "CASCADE"}
            ]
        },
        "performance_considerations": {
            "partitioning": [
                "posts table partitioned by created_at (weekly partitions)",
                "likes table partitioned by created_at (monthly partitions)",
                "comments table partitioned by created_at (monthly partitions)"
            ],
            "indexing_strategy": [
                "Composite indexes for timeline queries",
                "Partial indexes for active users only",
                "Covering indexes to avoid table lookups"
            ],
            "caching": [
                "Redis for user timeline feeds (15-minute TTL)",
                "CDN for media content",
                "Application cache for trending posts"
            ],
            "sharding": "posts and likes tables sharded by user_id hash",
            "read_replicas": "5 read replicas for feed generation",
            "search_optimization": "Elasticsearch for content search and discovery"
        },
        "queries": {
            "common_operations": [
                {
                    "description": "Generate user timeline feed",
                    "sql": "SELECT p.*, u.username, u.display_name, u.avatar_url FROM posts p JOIN users u ON p.user_id = u.id WHERE p.user_id IN (SELECT following_id FROM follows WHERE follower_id = ? AND status = 'accepted') AND p.visibility IN ('public', 'followers') ORDER BY p.engagement_score DESC, p.created_at DESC LIMIT 50",
                    "performance_notes": "Uses follow relationship index and post engagement index"
                },
                {
                    "description": "Get post with engagement metrics",
                    "sql": "SELECT p.*, u.username, u.display_name, u.avatar_url, (SELECT COUNT(*) FROM likes l WHERE l.post_id = p.id) as like_count, (SELECT COUNT(*) FROM comments c WHERE c.post_id = p.id) as comment_count FROM posts p JOIN users u ON p.user_id = u.id WHERE p.id = ?",
                    "performance_notes": "Consider denormalized like_count and comment_count columns"
                },
                {
                    "description": "Search posts by content",
                    "sql": "SELECT p.*, u.username, MATCH(p.content) AGAINST (? IN NATURAL LANGUAGE MODE) as relevance FROM posts p JOIN users u ON p.user_id = u.id WHERE MATCH(p.content) AGAINST (? IN NATURAL LANGUAGE MODE) AND p.visibility = 'public' ORDER BY relevance DESC, p.created_at DESC LIMIT 20",
                    "performance_notes": "Uses fulltext index on content column"
                }
            ],
            "analytics_queries": [
                {
                    "description": "Top engaging posts in last 24 hours",
                    "sql": "SELECT p.id, p.content, p.like_count, p.comment_count, p.share_count, p.engagement_score FROM posts p WHERE p.created_at >= DATE_SUB(NOW(), INTERVAL 24 HOUR) AND p.visibility = 'public' ORDER BY p.engagement_score DESC LIMIT 100",
                    "performance_notes": "Benefits from engagement score index and time partitioning"
                }
            ]
        }
    },

    {
        "id": "financial_trading_system",
        "business_requirements": "High-frequency trading platform with real-time market data, order management, portfolio tracking, risk management, and regulatory compliance. Handle millions of trades per day with sub-millisecond latency requirements.",
        "domain_context": "Financial services with strict regulatory requirements, real-time risk monitoring, and multi-asset class support (stocks, options, futures, forex).",
        "schema_design": {
            "tables": [
                {
                    "name": "users",
                    "columns": [
                        {"name": "id", "type": "BIGINT", "constraints": ["PRIMARY KEY", "AUTO_INCREMENT"]},
                        {"name": "user_type", "type": "ENUM('individual', 'institution', 'market_maker')", "constraints": ["NOT NULL"]},
                        {"name": "account_number", "type": "VARCHAR(20)", "constraints": ["UNIQUE", "NOT NULL"]},
                        {"name": "email", "type": "VARCHAR(255)", "constraints": ["UNIQUE", "NOT NULL"]},
                        {"name": "kyc_status", "type": "ENUM('pending', 'verified', 'rejected')", "constraints": ["DEFAULT 'pending'"]},
                        {"name": "risk_level", "type": "ENUM('conservative', 'moderate', 'aggressive')", "constraints": ["DEFAULT 'conservative'"]},
                        {"name": "max_daily_loss", "type": "DECIMAL(15,2)", "constraints": []},
                        {"name": "created_at", "type": "TIMESTAMP", "constraints": ["DEFAULT CURRENT_TIMESTAMP"]},
                        {"name": "last_login_at", "type": "TIMESTAMP", "constraints": []},
                        {"name": "is_active", "type": "BOOLEAN", "constraints": ["DEFAULT TRUE"]}
                    ],
                    "indexes": [
                        {"name": "idx_users_account", "columns": ["account_number"], "type": "UNIQUE"},
                        {"name": "idx_users_kyc", "columns": ["kyc_status"]},
                        {"name": "idx_users_type", "columns": ["user_type"]},
                        {"name": "idx_users_active", "columns": ["is_active"]}
                    ]
                },
                {
                    "name": "instruments",
                    "columns": [
                        {"name": "id", "type": "BIGINT", "constraints": ["PRIMARY KEY", "AUTO_INCREMENT"]},
                        {"name": "symbol", "type": "VARCHAR(20)", "constraints": ["UNIQUE", "NOT NULL"]},
                        {"name": "instrument_type", "type": "ENUM('stock', 'option', 'future', 'forex', 'crypto')", "constraints": ["NOT NULL"]},
                        {"name": "name", "type": "VARCHAR(255)", "constraints": ["NOT NULL"]},
                        {"name": "exchange", "type": "VARCHAR(10)", "constraints": ["NOT NULL"]},
                        {"name": "currency", "type": "CHAR(3)", "constraints": ["NOT NULL"]},
                        {"name": "tick_size", "type": "DECIMAL(10,6)", "constraints": ["NOT NULL"]},
                        {"name": "lot_size", "type": "INT", "constraints": ["DEFAULT 1"]},
                        {"name": "margin_requirement", "type": "DECIMAL(5,4)", "constraints": []},
                        {"name": "is_tradeable", "type": "BOOLEAN", "constraints": ["DEFAULT TRUE"]},
                        {"name": "created_at", "type": "TIMESTAMP", "constraints": ["DEFAULT CURRENT_TIMESTAMP"]}
                    ],
                    "indexes": [
                        {"name": "idx_instruments_symbol", "columns": ["symbol"], "type": "UNIQUE"},
                        {"name": "idx_instruments_type", "columns": ["instrument_type"]},
                        {"name": "idx_instruments_exchange", "columns": ["exchange"]},
                        {"name": "idx_instruments_tradeable", "columns": ["is_tradeable"]}
                    ]
                },
                {
                    "name": "orders",
                    "columns": [
                        {"name": "id", "type": "BIGINT", "constraints": ["PRIMARY KEY", "AUTO_INCREMENT"]},
                        {"name": "user_id", "type": "BIGINT", "constraints": ["NOT NULL"]},
                        {"name": "instrument_id", "type": "BIGINT", "constraints": ["NOT NULL"]},
                        {"name": "order_id", "type": "VARCHAR(50)", "constraints": ["UNIQUE", "NOT NULL"]},
                        {"name": "side", "type": "ENUM('buy', 'sell')", "constraints": ["NOT NULL"]},
                        {"name": "order_type", "type": "ENUM('market', 'limit', 'stop', 'stop_limit')", "constraints": ["NOT NULL"]},
                        {"name": "quantity", "type": "BIGINT", "constraints": ["NOT NULL"]},
                        {"name": "price", "type": "DECIMAL(15,6)", "constraints": []},
                        {"name": "stop_price", "type": "DECIMAL(15,6)", "constraints": []},
                        {"name": "filled_quantity", "type": "BIGINT", "constraints": ["DEFAULT 0"]},
                        {"name": "avg_fill_price", "type": "DECIMAL(15,6)", "constraints": []},
                        {"name": "status", "type": "ENUM('pending', 'partial', 'filled', 'cancelled', 'rejected')", "constraints": ["DEFAULT 'pending'"]},
                        {"name": "time_in_force", "type": "ENUM('day', 'gtc', 'ioc', 'fok')", "constraints": ["DEFAULT 'day'"]},
                        {"name": "created_at", "type": "TIMESTAMP(6)", "constraints": ["DEFAULT CURRENT_TIMESTAMP(6)"]},
                        {"name": "updated_at", "type": "TIMESTAMP(6)", "constraints": ["DEFAULT CURRENT_TIMESTAMP(6) ON UPDATE CURRENT_TIMESTAMP(6)"]},
                        {"name": "expires_at", "type": "TIMESTAMP", "constraints": []}
                    ],
                    "indexes": [
                        {"name": "idx_orders_user_status", "columns": ["user_id", "status"]},
                        {"name": "idx_orders_instrument_side", "columns": ["instrument_id", "side", "created_at"]},
                        {"name": "idx_orders_status_created", "columns": ["status", "created_at"]},
                        {"name": "idx_orders_price_quantity", "columns": ["instrument_id", "price", "quantity"]},
                        {"name": "idx_orders_expiry", "columns": ["expires_at"]}
                    ]
                },
                {
                    "name": "trades",
                    "columns": [
                        {"name": "id", "type": "BIGINT", "constraints": ["PRIMARY KEY", "AUTO_INCREMENT"]},
                        {"name": "trade_id", "type": "VARCHAR(50)", "constraints": ["UNIQUE", "NOT NULL"]},
                        {"name": "buy_order_id", "type": "BIGINT", "constraints": ["NOT NULL"]},
                        {"name": "sell_order_id", "type": "BIGINT", "constraints": ["NOT NULL"]},
                        {"name": "instrument_id", "type": "BIGINT", "constraints": ["NOT NULL"]},
                        {"name": "quantity", "type": "BIGINT", "constraints": ["NOT NULL"]},
                        {"name": "price", "type": "DECIMAL(15,6)", "constraints": ["NOT NULL"]},
                        {"name": "trade_value", "type": "DECIMAL(20,6)", "constraints": ["NOT NULL"]},
                        {"name": "commission", "type": "DECIMAL(10,6)", "constraints": ["DEFAULT 0"]},
                        {"name": "executed_at", "type": "TIMESTAMP(6)", "constraints": ["DEFAULT CURRENT_TIMESTAMP(6)"]},
                        {"name": "settlement_date", "type": "DATE", "constraints": ["NOT NULL"]}
                    ],
                    "indexes": [
                        {"name": "idx_trades_instrument_time", "columns": ["instrument_id", "executed_at"]},
                        {"name": "idx_trades_buy_order", "columns": ["buy_order_id"]},
                        {"name": "idx_trades_sell_order", "columns": ["sell_order_id"]},
                        {"name": "idx_trades_settlement", "columns": ["settlement_date"]},
                        {"name": "idx_trades_value_time", "columns": ["trade_value", "executed_at"]}
                    ]
                },
                {
                    "name": "portfolios",
                    "columns": [
                        {"name": "id", "type": "BIGINT", "constraints": ["PRIMARY KEY", "AUTO_INCREMENT"]},
                        {"name": "user_id", "type": "BIGINT", "constraints": ["NOT NULL"]},
                        {"name": "instrument_id", "type": "BIGINT", "constraints": ["NOT NULL"]},
                        {"name": "position", "type": "BIGINT", "constraints": ["DEFAULT 0"]},
                        {"name": "avg_cost", "type": "DECIMAL(15,6)", "constraints": ["DEFAULT 0"]},
                        {"name": "market_value", "type": "DECIMAL(20,6)", "constraints": ["DEFAULT 0"]},
                        {"name": "unrealized_pnl", "type": "DECIMAL(20,6)", "constraints": ["DEFAULT 0"]},
                        {"name": "realized_pnl", "type": "DECIMAL(20,6)", "constraints": ["DEFAULT 0"]},
                        {"name": "last_updated", "type": "TIMESTAMP(6)", "constraints": ["DEFAULT CURRENT_TIMESTAMP(6) ON UPDATE CURRENT_TIMESTAMP(6)"]}
                    ],
                    "indexes": [
                        {"name": "idx_portfolios_user_instrument", "columns": ["user_id", "instrument_id"], "type": "UNIQUE"},
                        {"name": "idx_portfolios_position", "columns": ["position"]},
                        {"name": "idx_portfolios_pnl", "columns": ["unrealized_pnl"]},
                        {"name": "idx_portfolios_updated", "columns": ["last_updated"]}
                    ]
                }
            ],
            "relationships": [
                {"from": "orders.user_id", "to": "users.id", "type": "FOREIGN KEY", "on_delete": "RESTRICT"},
                {"from": "orders.instrument_id", "to": "instruments.id", "type": "FOREIGN KEY", "on_delete": "RESTRICT"},
                {"from": "trades.buy_order_id", "to": "orders.id", "type": "FOREIGN KEY", "on_delete": "RESTRICT"},
                {"from": "trades.sell_order_id", "to": "orders.id", "type": "FOREIGN KEY", "on_delete": "RESTRICT"},
                {"from": "trades.instrument_id", "to": "instruments.id", "type": "FOREIGN KEY", "on_delete": "RESTRICT"},
                {"from": "portfolios.user_id", "to": "users.id", "type": "FOREIGN KEY", "on_delete": "CASCADE"},
                {"from": "portfolios.instrument_id", "to": "instruments.id", "type": "FOREIGN KEY", "on_delete": "RESTRICT"}
            ]
        },
        "performance_considerations": {
            "partitioning": [
                "orders table partitioned by created_at (daily partitions)",
                "trades table partitioned by executed_at (hourly partitions)",
                "market_data table partitioned by timestamp (minute partitions)"
            ],
            "indexing_strategy": [
                "Microsecond precision timestamps for trade ordering",
                "Covering indexes for order book queries",
                "Partial indexes for active orders only"
            ],
            "caching": [
                "Redis for real-time order book data",
                "In-memory cache for instrument metadata",
                "Application-level caching for user positions"
            ],
            "hardware_optimization": "SSD storage with battery-backed write cache",
            "connection_pooling": "Dedicated connection pools per trading strategy",
            "replication": "Synchronous replication to hot standby for zero data loss"
        },
        "queries": {
            "common_operations": [
                {
                    "description": "Get user's open orders",
                    "sql": "SELECT o.*, i.symbol, i.instrument_type FROM orders o JOIN instruments i ON o.instrument_id = i.id WHERE o.user_id = ? AND o.status IN ('pending', 'partial') ORDER BY o.created_at DESC",
                    "performance_notes": "Uses composite index on (user_id, status)"
                },
                {
                    "description": "Calculate portfolio value",
                    "sql": "SELECT p.*, i.symbol, (p.position * md.last_price) as current_value FROM portfolios p JOIN instruments i ON p.instrument_id = i.id JOIN market_data md ON i.id = md.instrument_id WHERE p.user_id = ? AND p.position != 0 AND md.timestamp = (SELECT MAX(timestamp) FROM market_data WHERE instrument_id = i.id)",
                    "performance_notes": "Requires real-time market data integration"
                },
                {
                    "description": "Risk monitoring - positions approaching limits",
                    "sql": "SELECT u.account_number, SUM(ABS(p.unrealized_pnl)) as total_loss FROM users u JOIN portfolios p ON u.id = p.user_id WHERE p.unrealized_pnl < 0 GROUP BY u.id HAVING total_loss > u.max_daily_loss * 0.8",
                    "performance_notes": "Uses portfolio PnL index for risk calculations"
                }
            ],
            "analytics_queries": [
                {
                    "description": "Trading volume by instrument",
                    "sql": "SELECT i.symbol, SUM(t.quantity) as volume, SUM(t.trade_value) as value, COUNT(*) as trade_count FROM trades t JOIN instruments i ON t.instrument_id = i.id WHERE t.executed_at >= DATE_SUB(NOW(), INTERVAL 1 DAY) GROUP BY i.id ORDER BY value DESC",
                    "performance_notes": "Benefits from time-based partitioning on trades table"
                }
            ]
        }
    },

    {
        "id": "healthcare_management_system",
        "business_requirements": "Electronic health records (EHR) system for hospitals and clinics managing patient records, appointments, medical history, prescriptions, billing, and insurance claims. HIPAA compliance required.",
        "domain_context": "Healthcare industry with strict privacy regulations, complex medical workflows, integration with insurance systems, and 24/7 availability requirements.",
        "schema_design": {
            "tables": [
                {
                    "name": "patients",
                    "columns": [
                        {"name": "id", "type": "BIGINT", "constraints": ["PRIMARY KEY", "AUTO_INCREMENT"]},
                        {"name": "medical_record_number", "type": "VARCHAR(20)", "constraints": ["UNIQUE", "NOT NULL"]},
                        {"name": "first_name", "type": "VARCHAR(100)", "constraints": ["NOT NULL"]},
                        {"name": "last_name", "type": "VARCHAR(100)", "constraints": ["NOT NULL"]},
                        {"name": "date_of_birth", "type": "DATE", "constraints": ["NOT NULL"]},
                        {"name": "gender", "type": "ENUM('M', 'F', 'O')", "constraints": []},
                        {"name": "ssn_hash", "type": "VARCHAR(256)", "constraints": ["UNIQUE"]},
                        {"name": "phone", "type": "VARCHAR(15)", "constraints": []},
                        {"name": "email", "type": "VARCHAR(255)", "constraints": []},
                        {"name": "emergency_contact", "type": "JSON", "constraints": []},
                        {"name": "insurance_info", "type": "JSON", "constraints": []},
                        {"name": "blood_type", "type": "VARCHAR(5)", "constraints": []},
                        {"name": "allergies", "type": "TEXT", "constraints": []},
                        {"name": "created_at", "type": "TIMESTAMP", "constraints": ["DEFAULT CURRENT_TIMESTAMP"]},
                        {"name": "updated_at", "type": "TIMESTAMP", "constraints": ["DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP"]},
                        {"name": "is_active", "type": "BOOLEAN", "constraints": ["DEFAULT TRUE"]}
                    ],
                    "indexes": [
                        {"name": "idx_patients_mrn", "columns": ["medical_record_number"], "type": "UNIQUE"},
                        {"name": "idx_patients_name", "columns": ["last_name", "first_name"]},
                        {"name": "idx_patients_dob", "columns": ["date_of_birth"]},
                        {"name": "idx_patients_ssn", "columns": ["ssn_hash"], "type": "UNIQUE"},
                        {"name": "idx_patients_active", "columns": ["is_active"]}
                    ]
                },
                {
                    "name": "healthcare_providers",
                    "columns": [
                        {"name": "id", "type": "BIGINT", "constraints": ["PRIMARY KEY", "AUTO_INCREMENT"]},
                        {"name": "npi_number", "type": "VARCHAR(10)", "constraints": ["UNIQUE", "NOT NULL"]},
                        {"name": "first_name", "type": "VARCHAR(100)", "constraints": ["NOT NULL"]},
                        {"name": "last_name", "type": "VARCHAR(100)", "constraints": ["NOT NULL"]},
                        {"name": "specialty", "type": "VARCHAR(100)", "constraints": ["NOT NULL"]},
                        {"name": "license_number", "type": "VARCHAR(50)", "constraints": ["UNIQUE", "NOT NULL"]},
                        {"name": "department_id", "type": "BIGINT", "constraints": []},
                        {"name": "phone", "type": "VARCHAR(15)", "constraints": []},
                        {"name": "email", "type": "VARCHAR(255)", "constraints": []},
                        {"name": "schedule_template", "type": "JSON", "constraints": []},
                        {"name": "is_accepting_patients", "type": "BOOLEAN", "constraints": ["DEFAULT TRUE"]},
                        {"name": "created_at", "type": "TIMESTAMP", "constraints": ["DEFAULT CURRENT_TIMESTAMP"]},
                        {"name": "is_active", "type": "BOOLEAN", "constraints": ["DEFAULT TRUE"]}
                    ],
                    "indexes": [
                        {"name": "idx_providers_npi", "columns": ["npi_number"], "type": "UNIQUE"},
                        {"name": "idx_providers_license", "columns": ["license_number"], "type": "UNIQUE"},
                        {"name": "idx_providers_specialty", "columns": ["specialty"]},
                        {"name": "idx_providers_department", "columns": ["department_id"]},
                        {"name": "idx_providers_accepting", "columns": ["is_accepting_patients"]}
                    ]
                },
                {
                    "name": "appointments",
                    "columns": [
                        {"name": "id", "type": "BIGINT", "constraints": ["PRIMARY KEY", "AUTO_INCREMENT"]},
                        {"name": "patient_id", "type": "BIGINT", "constraints": ["NOT NULL"]},
                        {"name": "provider_id", "type": "BIGINT", "constraints": ["NOT NULL"]},
                        {"name": "appointment_type", "type": "ENUM('consultation', 'follow_up', 'procedure', 'emergency')", "constraints": ["NOT NULL"]},
                        {"name": "scheduled_at", "type": "DATETIME", "constraints": ["NOT NULL"]},
                        {"name": "duration_minutes", "type": "INT", "constraints": ["DEFAULT 30"]},
                        {"name": "status", "type": "ENUM('scheduled', 'confirmed', 'checked_in', 'in_progress', 'completed', 'cancelled', 'no_show')", "constraints": ["DEFAULT 'scheduled'"]},
                        {"name": "chief_complaint", "type": "TEXT", "constraints": []},
                        {"name": "notes", "type": "TEXT", "constraints": []},
                        {"name": "room_number", "type": "VARCHAR(10)", "constraints": []},
                        {"name": "created_at", "type": "TIMESTAMP", "constraints": ["DEFAULT CURRENT_TIMESTAMP"]},
                        {"name": "updated_at", "type": "TIMESTAMP", "constraints": ["DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP"]}
                    ],
                    "indexes": [
                        {"name": "idx_appointments_patient", "columns": ["patient_id", "scheduled_at"]},
                        {"name": "idx_appointments_provider", "columns": ["provider_id", "scheduled_at"]},
                        {"name": "idx_appointments_status", "columns": ["status", "scheduled_at"]},
                        {"name": "idx_appointments_room", "columns": ["room_number", "scheduled_at"]},
                        {"name": "idx_appointments_date", "columns": ["scheduled_at"]}
                    ]
                },
                {
                    "name": "medical_records",
                    "columns": [
                        {"name": "id", "type": "BIGINT", "constraints": ["PRIMARY KEY", "AUTO_INCREMENT"]},
                        {"name": "patient_id", "type": "BIGINT", "constraints": ["NOT NULL"]},
                        {"name": "provider_id", "type": "BIGINT", "constraints": ["NOT NULL"]},
                        {"name": "appointment_id", "type": "BIGINT", "constraints": []},
                        {"name": "record_type", "type": "ENUM('visit_note', 'diagnosis', 'lab_result', 'imaging', 'prescription')", "constraints": ["NOT NULL"]},
                        {"name": "icd_10_codes", "type": "JSON", "constraints": []},
                        {"name": "cpt_codes", "type": "JSON", "constraints": []},
                        {"name": "chief_complaint", "type": "TEXT", "constraints": []},
                        {"name": "history_of_present_illness", "type": "TEXT", "constraints": []},
                        {"name": "physical_examination", "type": "TEXT", "constraints": []},
                        {"name": "assessment", "type": "TEXT", "constraints": []},
                        {"name": "plan", "type": "TEXT", "constraints": []},
                        {"name": "vital_signs", "type": "JSON", "constraints": []},
                        {"name": "medications", "type": "JSON", "constraints": []},
                        {"name": "created_at", "type": "TIMESTAMP", "constraints": ["DEFAULT CURRENT_TIMESTAMP"]},
                        {"name": "updated_at", "type": "TIMESTAMP", "constraints": ["DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP"]},
                        {"name": "signed_at", "type": "TIMESTAMP", "constraints": []},
                        {"name": "is_signed", "type": "BOOLEAN", "constraints": ["DEFAULT FALSE"]}
                    ],
                    "indexes": [
                        {"name": "idx_medical_records_patient", "columns": ["patient_id", "created_at"]},
                        {"name": "idx_medical_records_provider", "columns": ["provider_id", "created_at"]},
                        {"name": "idx_medical_records_type", "columns": ["record_type", "created_at"]},
                        {"name": "idx_medical_records_appointment", "columns": ["appointment_id"]},
                        {"name": "idx_medical_records_signed", "columns": ["is_signed", "signed_at"]}
                    ]
                },
                {
                    "name": "prescriptions",
                    "columns": [
                        {"name": "id", "type": "BIGINT", "constraints": ["PRIMARY KEY", "AUTO_INCREMENT"]},
                        {"name": "patient_id", "type": "BIGINT", "constraints": ["NOT NULL"]},
                        {"name": "provider_id", "type": "BIGINT", "constraints": ["NOT NULL"]},
                        {"name": "medical_record_id", "type": "BIGINT", "constraints": []},
                        {"name": "medication_name", "type": "VARCHAR(255)", "constraints": ["NOT NULL"]},
                        {"name": "dosage", "type": "VARCHAR(100)", "constraints": ["NOT NULL"]},
                        {"name": "frequency", "type": "VARCHAR(100)", "constraints": ["NOT NULL"]},
                        {"name": "duration", "type": "VARCHAR(100)", "constraints": []},
                        {"name": "quantity", "type": "INT", "constraints": []},
                        {"name": "refills", "type": "INT", "constraints": ["DEFAULT 0"]},
                        {"name": "instructions", "type": "TEXT", "constraints": []},
                        {"name": "ndc_code", "type": "VARCHAR(20)", "constraints": []},
                        {"name": "status", "type": "ENUM('active', 'completed', 'cancelled', 'expired')", "constraints": ["DEFAULT 'active'"]},
                        {"name": "prescribed_at", "type": "TIMESTAMP", "constraints": ["DEFAULT CURRENT_TIMESTAMP"]},
                        {"name": "expires_at", "type": "TIMESTAMP", "constraints": []},
                        {"name": "pharmacy_id", "type": "BIGINT", "constraints": []}
                    ],
                    "indexes": [
                        {"name": "idx_prescriptions_patient", "columns": ["patient_id", "prescribed_at"]},
                        {"name": "idx_prescriptions_provider", "columns": ["provider_id", "prescribed_at"]},
                        {"name": "idx_prescriptions_status", "columns": ["status", "expires_at"]},
                        {"name": "idx_prescriptions_medication", "columns": ["medication_name"]},
                        {"name": "idx_prescriptions_ndc", "columns": ["ndc_code"]}
                    ]
                }
            ],
            "relationships": [
                {"from": "appointments.patient_id", "to": "patients.id", "type": "FOREIGN KEY", "on_delete": "RESTRICT"},
                {"from": "appointments.provider_id", "to": "healthcare_providers.id", "type": "FOREIGN KEY", "on_delete": "RESTRICT"},
                {"from": "medical_records.patient_id", "to": "patients.id", "type": "FOREIGN KEY", "on_delete": "RESTRICT"},
                {"from": "medical_records.provider_id", "to": "healthcare_providers.id", "type": "FOREIGN KEY", "on_delete": "RESTRICT"},
                {"from": "medical_records.appointment_id", "to": "appointments.id", "type": "FOREIGN KEY", "on_delete": "SET NULL"},
                {"from": "prescriptions.patient_id", "to": "patients.id", "type": "FOREIGN KEY", "on_delete": "RESTRICT"},
                {"from": "prescriptions.provider_id", "to": "healthcare_providers.id", "type": "FOREIGN KEY", "on_delete": "RESTRICT"},
                {"from": "prescriptions.medical_record_id", "to": "medical_records.id", "type": "FOREIGN KEY", "on_delete": "SET NULL"}
            ]
        },
        "performance_considerations": {
            "partitioning": [
                "medical_records table partitioned by created_at (monthly partitions)",
                "appointments table partitioned by scheduled_at (quarterly partitions)",
                "audit_logs table partitioned by created_at (weekly partitions)"
            ],
            "indexing_strategy": [
                "Composite indexes for patient timeline queries",
                "Partial indexes for active appointments only",
                "Covering indexes for appointment scheduling"
            ],
            "security": [
                "Encryption at rest for all patient data",
                "Row-level security for provider access",
                "Audit logging for all data access"
            ],
            "backup_strategy": "Continuous backup with 15-minute RPO",
            "compliance": "HIPAA-compliant data handling and access controls"
        },
        "queries": {
            "common_operations": [
                {
                    "description": "Get patient's complete medical history",
                    "sql": "SELECT mr.*, hp.first_name, hp.last_name, hp.specialty FROM medical_records mr JOIN healthcare_providers hp ON mr.provider_id = hp.id WHERE mr.patient_id = ? ORDER BY mr.created_at DESC LIMIT 50",
                    "performance_notes": "Uses composite index on (patient_id, created_at)"
                },
                {
                    "description": "Provider's daily schedule",
                    "sql": "SELECT a.*, p.first_name, p.last_name, p.medical_record_number FROM appointments a JOIN patients p ON a.patient_id = p.id WHERE a.provider_id = ? AND DATE(a.scheduled_at) = CURDATE() AND a.status NOT IN ('cancelled', 'no_show') ORDER BY a.scheduled_at",
                    "performance_notes": "Uses composite index on (provider_id, scheduled_at)"
                },
                {
                    "description": "Active prescriptions for patient",
                    "sql": "SELECT pr.*, hp.first_name as provider_name, hp.last_name as provider_lastname FROM prescriptions pr JOIN healthcare_providers hp ON pr.provider_id = hp.id WHERE pr.patient_id = ? AND pr.status = 'active' AND (pr.expires_at IS NULL OR pr.expires_at > NOW()) ORDER BY pr.prescribed_at DESC",
                    "performance_notes": "Uses composite index on (patient_id, status, expires_at)"
                }
            ],
            "analytics_queries": [
                {
                    "description": "Most common diagnoses by specialty",
                    "sql": "SELECT hp.specialty, JSON_UNQUOTE(JSON_EXTRACT(mr.icd_10_codes, '$[0].code')) as icd_code, COUNT(*) as frequency FROM medical_records mr JOIN healthcare_providers hp ON mr.provider_id = hp.id WHERE mr.icd_10_codes IS NOT NULL AND mr.created_at >= DATE_SUB(NOW(), INTERVAL 1 YEAR) GROUP BY hp.specialty, icd_code ORDER BY frequency DESC",
                    "performance_notes": "Benefits from JSON indexing on ICD-10 codes"
                }
            ]
        }
    },

    {
        "id": "educational_lms_platform",
        "business_requirements": "Learning Management System for universities and online education platforms. Support course management, student enrollment, assignments, grading, discussions, and analytics. Handle 100K+ students and 10K+ courses.",
        "domain_context": "EdTech platform with complex academic hierarchies, semester-based scheduling, multi-role access, and comprehensive reporting requirements.",
        "schema_design": {
            "tables": [
                {
                    "name": "institutions",
                    "columns": [
                        {"name": "id", "type": "BIGINT", "constraints": ["PRIMARY KEY", "AUTO_INCREMENT"]},
                        {"name": "name", "type": "VARCHAR(255)", "constraints": ["NOT NULL"]},
                        {"name": "code", "type": "VARCHAR(20)", "constraints": ["UNIQUE", "NOT NULL"]},
                        {"name": "type", "type": "ENUM('university', 'college', 'school', 'corporate')", "constraints": ["NOT NULL"]},
                        {"name": "address", "type": "JSON", "constraints": []},
                        {"name": "contact_info", "type": "JSON", "constraints": []},
                        {"name": "settings", "type": "JSON", "constraints": []},
                        {"name": "created_at", "type": "TIMESTAMP", "constraints": ["DEFAULT CURRENT_TIMESTAMP"]},
                        {"name": "is_active", "type": "BOOLEAN", "constraints": ["DEFAULT TRUE"]}
                    ],
                    "indexes": [
                        {"name": "idx_institutions_code", "columns": ["code"], "type": "UNIQUE"},
                        {"name": "idx_institutions_type", "columns": ["type"]},
                        {"name": "idx_institutions_active", "columns": ["is_active"]}
                    ]
                },
                {
                    "name": "users",
                    "columns": [
                        {"name": "id", "type": "BIGINT", "constraints": ["PRIMARY KEY", "AUTO_INCREMENT"]},
                        {"name": "institution_id", "type": "BIGINT", "constraints": ["NOT NULL"]},
                        {"name": "user_id", "type": "VARCHAR(50)", "constraints": ["NOT NULL"]},
                        {"name": "email", "type": "VARCHAR(255)", "constraints": ["NOT NULL"]},
                        {"name": "first_name", "type": "VARCHAR(100)", "constraints": ["NOT NULL"]},
                        {"name": "last_name", "type": "VARCHAR(100)", "constraints": ["NOT NULL"]},
                        {"name": "role", "type": "ENUM('student', 'instructor', 'admin', 'ta')", "constraints": ["NOT NULL"]},
                        {"name": "department", "type": "VARCHAR(100)", "constraints": []},
                        {"name": "year_level", "type": "INT", "constraints": []},
                        {"name": "major", "type": "VARCHAR(100)", "constraints": []},
                        {"name": "gpa", "type": "DECIMAL(3,2)", "constraints": []},
                        {"name": "profile_data", "type": "JSON", "constraints": []},
                        {"name": "last_login", "type": "TIMESTAMP", "constraints": []},
                        {"name": "created_at", "type": "TIMESTAMP", "constraints": ["DEFAULT CURRENT_TIMESTAMP"]},
                        {"name": "is_active", "type": "BOOLEAN", "constraints": ["DEFAULT TRUE"]}
                    ],
                    "indexes": [
                        {"name": "idx_users_institution_userid", "columns": ["institution_id", "user_id"], "type": "UNIQUE"},
                        {"name": "idx_users_email", "columns": ["email"], "type": "UNIQUE"},
                        {"name": "idx_users_role", "columns": ["role"]},
                        {"name": "idx_users_department", "columns": ["department"]},
                        {"name": "idx_users_major", "columns": ["major"]}
                    ]
                },
                {
                    "name": "courses",
                    "columns": [
                        {"name": "id", "type": "BIGINT", "constraints": ["PRIMARY KEY", "AUTO_INCREMENT"]},
                        {"name": "institution_id", "type": "BIGINT", "constraints": ["NOT NULL"]},
                        {"name": "course_code", "type": "VARCHAR(20)", "constraints": ["NOT NULL"]},
                        {"name": "title", "type": "VARCHAR(255)", "constraints": ["NOT NULL"]},
                        {"name": "description", "type": "TEXT", "constraints": []},
                        {"name": "department", "type": "VARCHAR(100)", "constraints": ["NOT NULL"]},
                        {"name": "credits", "type": "INT", "constraints": ["DEFAULT 3"]},
                        {"name": "level", "type": "ENUM('undergraduate', 'graduate', 'doctoral')", "constraints": ["NOT NULL"]},
                        {"name": "prerequisites", "type": "JSON", "constraints": []},
                        {"name": "learning_objectives", "type": "JSON", "constraints": []},
                        {"name": "syllabus_url", "type": "VARCHAR(500)", "constraints": []},
                        {"name": "created_at", "type": "TIMESTAMP", "constraints": ["DEFAULT CURRENT_TIMESTAMP"]},
                        {"name": "is_active", "type": "BOOLEAN", "constraints": ["DEFAULT TRUE"]}
                    ],
                    "indexes": [
                        {"name": "idx_courses_institution_code", "columns": ["institution_id", "course_code"], "type": "UNIQUE"},
                        {"name": "idx_courses_department", "columns": ["department"]},
                        {"name": "idx_courses_level", "columns": ["level"]},
                        {"name": "idx_courses_credits", "columns": ["credits"]},
                        {"name": "idx_courses_search", "columns": ["title", "description"], "type": "FULLTEXT"}
                    ]
                },
                {
                    "name": "course_sections",
                    "columns": [
                        {"name": "id", "type": "BIGINT", "constraints": ["PRIMARY KEY", "AUTO_INCREMENT"]},
                        {"name": "course_id", "type": "BIGINT", "constraints": ["NOT NULL"]},
                        {"name": "instructor_id", "type": "BIGINT", "constraints": ["NOT NULL"]},
                        {"name": "section_number", "type": "VARCHAR(10)", "constraints": ["NOT NULL"]},
                        {"name": "semester", "type": "VARCHAR(20)", "constraints": ["NOT NULL"]},
                        {"name": "year", "type": "INT", "constraints": ["NOT NULL"]},
                        {"name": "max_enrollment", "type": "INT", "constraints": ["NOT NULL"]},
                        {"name": "current_enrollment", "type": "INT", "constraints": ["DEFAULT 0"]},
                        {"name": "meeting_times", "type": "JSON", "constraints": []},
                        {"name": "classroom", "type": "VARCHAR(50)", "constraints": []},
                        {"name": "start_date", "type": "DATE", "constraints": ["NOT NULL"]},
                        {"name": "end_date", "type": "DATE", "constraints": ["NOT NULL"]},
                        {"name": "status", "type": "ENUM('planning', 'open', 'closed', 'active', 'completed')", "constraints": ["DEFAULT 'planning'"]},
                        {"name": "created_at", "type": "TIMESTAMP", "constraints": ["DEFAULT CURRENT_TIMESTAMP"]}
                    ],
                    "indexes": [
                        {"name": "idx_sections_course_semester", "columns": ["course_id", "semester", "year"]},
                        {"name": "idx_sections_instructor", "columns": ["instructor_id", "semester", "year"]},
                        {"name": "idx_sections_enrollment", "columns": ["status", "current_enrollment", "max_enrollment"]},
                        {"name": "idx_sections_dates", "columns": ["start_date", "end_date"]},
                        {"name": "idx_sections_classroom", "columns": ["classroom", "semester"]}
                    ]
                },
                {
                    "name": "enrollments",
                    "columns": [
                        {"name": "id", "type": "BIGINT", "constraints": ["PRIMARY KEY", "AUTO_INCREMENT"]},
                        {"name": "student_id", "type": "BIGINT", "constraints": ["NOT NULL"]},
                        {"name": "section_id", "type": "BIGINT", "constraints": ["NOT NULL"]},
                        {"name": "enrollment_status", "type": "ENUM('enrolled', 'waitlisted', 'dropped', 'withdrawn')", "constraints": ["DEFAULT 'enrolled'"]},
                        {"name": "grade", "type": "VARCHAR(5)", "constraints": []},
                        {"name": "grade_points", "type": "DECIMAL(3,2)", "constraints": []},
                        {"name": "attendance_percentage", "type": "DECIMAL(5,2)", "constraints": []},
                        {"name": "enrollment_date", "type": "TIMESTAMP", "constraints": ["DEFAULT CURRENT_TIMESTAMP"]},
                        {"name": "completion_date", "type": "TIMESTAMP", "constraints": []},
                        {"name": "notes", "type": "TEXT", "constraints": []}
                    ],
                    "indexes": [
                        {"name": "idx_enrollments_student_section", "columns": ["student_id", "section_id"], "type": "UNIQUE"},
                        {"name": "idx_enrollments_section_status", "columns": ["section_id", "enrollment_status"]},
                        {"name": "idx_enrollments_student_date", "columns": ["student_id", "enrollment_date"]},
                        {"name": "idx_enrollments_grade", "columns": ["grade", "grade_points"]},
                        {"name": "idx_enrollments_completion", "columns": ["completion_date"]}
                    ]
                },
                {
                    "name": "assignments",
                    "columns": [
                        {"name": "id", "type": "BIGINT", "constraints": ["PRIMARY KEY", "AUTO_INCREMENT"]},
                        {"name": "section_id", "type": "BIGINT", "constraints": ["NOT NULL"]},
                        {"name": "title", "type": "VARCHAR(255)", "constraints": ["NOT NULL"]},
                        {"name": "description", "type": "TEXT", "constraints": []},
                        {"name": "type", "type": "ENUM('homework', 'quiz', 'exam', 'project', 'discussion')", "constraints": ["NOT NULL"]},
                        {"name": "max_points", "type": "DECIMAL(8,2)", "constraints": ["NOT NULL"]},
                        {"name": "weight", "type": "DECIMAL(5,4)", "constraints": ["DEFAULT 1.0000"]},
                        {"name": "due_date", "type": "DATETIME", "constraints": []},
                        {"name": "available_from", "type": "DATETIME", "constraints": []},
                        {"name": "available_until", "type": "DATETIME", "constraints": []},
                        {"name": "submission_types", "type": "JSON", "constraints": []},
                        {"name": "rubric", "type": "JSON", "constraints": []},
                        {"name": "instructions", "type": "TEXT", "constraints": []},
                        {"name": "attachments", "type": "JSON", "constraints": []},
                        {"name": "allow_late_submission", "type": "BOOLEAN", "constraints": ["DEFAULT FALSE"]},
                        {"name": "late_penalty", "type": "DECIMAL(5,4)", "constraints": []},
                        {"name": "created_at", "type": "TIMESTAMP", "constraints": ["DEFAULT CURRENT_TIMESTAMP"]},
                        {"name": "updated_at", "type": "TIMESTAMP", "constraints": ["DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP"]},
                        {"name": "is_published", "type": "BOOLEAN", "constraints": ["DEFAULT FALSE"]}
                    ],
                    "indexes": [
                        {"name": "idx_assignments_section", "columns": ["section_id", "due_date"]},
                        {"name": "idx_assignments_type", "columns": ["type", "due_date"]},
                        {"name": "idx_assignments_published", "columns": ["is_published", "available_from"]},
                        {"name": "idx_assignments_due_date", "columns": ["due_date"]},
                        {"name": "idx_assignments_weight", "columns": ["weight", "max_points"]}
                    ]
                },
                {
                    "name": "submissions",
                    "columns": [
                        {"name": "id", "type": "BIGINT", "constraints": ["PRIMARY KEY", "AUTO_INCREMENT"]},
                        {"name": "assignment_id", "type": "BIGINT", "constraints": ["NOT NULL"]},
                        {"name": "student_id", "type": "BIGINT", "constraints": ["NOT NULL"]},
                        {"name": "submission_text", "type": "LONGTEXT", "constraints": []},
                        {"name": "attachments", "type": "JSON", "constraints": []},
                        {"name": "submitted_at", "type": "TIMESTAMP", "constraints": ["NOT NULL"]},
                        {"name": "is_late", "type": "BOOLEAN", "constraints": ["DEFAULT FALSE"]},
                        {"name": "score", "type": "DECIMAL(8,2)", "constraints": []},
                        {"name": "feedback", "type": "TEXT", "constraints": []},
                        {"name": "graded_by", "type": "BIGINT", "constraints": []},
                        {"name": "graded_at", "type": "TIMESTAMP", "constraints": []},
                        {"name": "status", "type": "ENUM('submitted', 'graded', 'returned', 'resubmitted')", "constraints": ["DEFAULT 'submitted'"]},
                        {"name": "attempt_number", "type": "INT", "constraints": ["DEFAULT 1"]},
                        {"name": "plagiarism_score", "type": "DECIMAL(5,2)", "constraints": []},
                        {"name": "created_at", "type": "TIMESTAMP", "constraints": ["DEFAULT CURRENT_TIMESTAMP"]},
                        {"name": "updated_at", "type": "TIMESTAMP", "constraints": ["DEFAULT CURRENT_TIMESTAMP ON UPDATE CURRENT_TIMESTAMP"]}
                    ],
                    "indexes": [
                        {"name": "idx_submissions_assignment_student", "columns": ["assignment_id", "student_id", "attempt_number"], "type": "UNIQUE"},
                        {"name": "idx_submissions_student", "columns": ["student_id", "submitted_at"]},
                        {"name": "idx_submissions_grading", "columns": ["status", "graded_at"]},
                        {"name": "idx_submissions_late", "columns": ["is_late", "submitted_at"]},
                        {"name": "idx_submissions_plagiarism", "columns": ["plagiarism_score"]}
                    ]
                }
            ],
            "relationships": [
                {"from": "users.institution_id", "to": "institutions.id", "type": "FOREIGN KEY", "on_delete": "RESTRICT"},
                {"from": "courses.institution_id", "to": "institutions.id", "type": "FOREIGN KEY", "on_delete": "RESTRICT"},
                {"from": "course_sections.course_id", "to": "courses.id", "type": "FOREIGN KEY", "on_delete": "CASCADE"},
                {"from": "course_sections.instructor_id", "to": "users.id", "type": "FOREIGN KEY", "on_delete": "RESTRICT"},
                {"from": "enrollments.student_id", "to": "users.id", "type": "FOREIGN KEY", "on_delete": "RESTRICT"},
                {"from": "enrollments.section_id", "to": "course_sections.id", "type": "FOREIGN KEY", "on_delete": "CASCADE"},
                {"from": "assignments.section_id", "to": "course_sections.id", "type": "FOREIGN KEY", "on_delete": "CASCADE"},
                {"from": "submissions.assignment_id", "to": "assignments.id", "type": "FOREIGN KEY", "on_delete": "CASCADE"},
                {"from": "submissions.student_id", "to": "users.id", "type": "FOREIGN KEY", "on_delete": "RESTRICT"},
                {"from": "submissions.graded_by", "to": "users.id", "type": "FOREIGN KEY", "on_delete": "SET NULL"}
            ]
        },
        "performance_considerations": {
            "partitioning": [
                "submissions table partitioned by submitted_at (semester partitions)",
                "enrollments table partitioned by enrollment_date (yearly partitions)",
                "activity_logs table partitioned by created_at (monthly partitions)"
            ],
            "indexing_strategy": [
                "Composite indexes for gradebook queries",
                "Partial indexes for active enrollments only",
                "JSON indexes for flexible course metadata"
            ],
            "caching": [
                "Redis for course catalogs and schedules",
                "Application cache for user permissions",
                "CDN for course materials and videos"
            ],
            "read_replicas": "3 read replicas for reporting and analytics",
            "archive_strategy": "Archive completed courses older than 7 years"
        },
        "queries": {
            "common_operations": [
                {
                    "description": "Student's current course schedule",
                    "sql": "SELECT cs.*, c.title, c.course_code, c.credits, u.first_name as instructor_name, u.last_name as instructor_lastname FROM enrollments e JOIN course_sections cs ON e.section_id = cs.id JOIN courses c ON cs.course_id = c.id JOIN users u ON cs.instructor_id = u.id WHERE e.student_id = ? AND e.enrollment_status = 'enrolled' AND cs.status = 'active' ORDER BY c.course_code",
                    "performance_notes": "Uses composite index on (student_id, enrollment_status)"
                },
                {
                    "description": "Assignment gradebook for instructor",
                    "sql": "SELECT a.title, a.max_points, s.score, s.submitted_at, s.is_late, u.first_name, u.last_name FROM assignments a LEFT JOIN submissions s ON a.id = s.assignment_id LEFT JOIN users u ON s.student_id = u.id WHERE a.section_id = ? ORDER BY u.last_name, u.first_name, a.due_date",
                    "performance_notes": "Uses section index and considers covering index for performance"
                },
                {
                    "description": "Course availability for enrollment",
                    "sql": "SELECT cs.*, c.title, c.course_code, c.credits, (cs.max_enrollment - cs.current_enrollment) as available_spots FROM course_sections cs JOIN courses c ON cs.course_id = c.id WHERE cs.status = 'open' AND cs.current_enrollment < cs.max_enrollment AND cs.semester = ? AND cs.year = ? ORDER BY c.department, c.course_code",
                    "performance_notes": "Uses composite index on (status, semester, year)"
                }
            ],
            "analytics_queries": [
                {
                    "description": "Grade distribution by course",
                    "sql": "SELECT c.course_code, c.title, e.grade, COUNT(*) as count, AVG(e.grade_points) as avg_gpa FROM enrollments e JOIN course_sections cs ON e.section_id = cs.id JOIN courses c ON cs.course_id = c.id WHERE cs.semester = ? AND cs.year = ? AND e.grade IS NOT NULL GROUP BY c.id, e.grade ORDER BY c.course_code, e.grade",
                    "performance_notes": "Benefits from semester partitioning and grade indexes"
                }
            ]
        }
    }
]

# 4. Define JSON Schema

We enforce structure via jsonschema.

In [ ]:
from jsonschema import Draft7Validator

schema = {
    "type": "object",
    "required": ["id", "business_requirements", "domain_context", "schema_design", "performance_considerations", "queries"],
    "properties": {
        "id": {
            "type": "string",
            "description": "Unique identifier for the use case"
        },
        "business_requirements": {
            "type": "string",
            "description": "Natural language description of the business requirements and scale"
        },
        "domain_context": {
            "type": "string",
            "description": "Domain-specific context and constraints"
        },
        "schema_design": {
            "type": "object",
            "required": ["tables", "relationships"],
            "properties": {
                "tables": {
                    "type": "array",
                    "items": {
                        "type": "object",
                        "required": ["name", "columns", "indexes"],
                        "properties": {
                            "name": {
                                "type": "string",
                                "description": "Table name"
                            },
                            "columns": {
                                "type": "array",
                                "items": {
                                    "type": "object",
                                    "required": ["name", "type", "constraints"],
                                    "properties": {
                                        "name": {"type": "string"},
                                        "type": {"type": "string"},
                                        "constraints": {
                                            "type": "array",
                                            "items": {"type": "string"}
                                        }
                                    }
                                }
                            },
                            "indexes": {
                                "type": "array",
                                "items": {
                                    "type": "object",
                                    "required": ["name", "columns"],
                                    "properties": {
                                        "name": {"type": "string"},
                                        "columns": {
                                            "type": "array",
                                            "items": {"type": "string"}
                                        },
                                        "type": {
                                            "type": "string",
                                            "enum": ["UNIQUE", "FULLTEXT", "BTREE", "HASH"]
                                        }
                                    }
                                }
                            }
                        }
                    }
                },
                "relationships": {
                    "type": "array",
                    "items": {
                        "type": "object",
                        "required": ["from", "to", "type"],
                        "properties": {
                            "from": {"type": "string"},
                            "to": {"type": "string"},
                            "type": {"type": "string"},
                            "on_delete": {
                                "type": "string",
                                "enum": ["CASCADE", "RESTRICT", "SET NULL", "NO ACTION"]
                            }
                        }
                    }
                }
            }
        },
        "performance_considerations": {
            "type": "object",
            "properties": {
                "partitioning": {
                    "type": "array",
                    "items": {"type": "string"}
                },
                "indexing_strategy": {
                    "type": "array",
                    "items": {"type": "string"}
                },
                "caching": {
                    "type": "array",
                    "items": {"type": "string"}
                },
                "read_replicas": {"type": "string"},
                "connection_pooling": {"type": "string"},
                "hardware_optimization": {"type": "string"},
                "replication": {"type": "string"},
                "sharding": {"type": "string"},
                "search_optimization": {"type": "string"},
                "backup_strategy": {"type": "string"},
                "compliance": {"type": "string"},
                "security": {
                    "type": "array",
                    "items": {"type": "string"}
                },
                "storage": {"type": "string"},
                "analytics": {"type": "string"}
            }
        },
        "queries": {
            "type": "object",
            "required": ["common_operations"],
            "properties": {
                "common_operations": {
                    "type": "array",
                    "items": {
                        "type": "object",
                        "required": ["description", "sql", "performance_notes"],
                        "properties": {
                            "description": {"type": "string"},
                            "sql": {"type": "string"},
                            "performance_notes": {"type": "string"}
                        }
                    }
                },
                "analytics_queries": {
                    "type": "array",
                    "items": {
                        "type": "object",
                        "required": ["description", "sql", "performance_notes"],
                        "properties": {
                            "description": {"type": "string"},
                            "sql": {"type": "string"},
                            "performance_notes": {"type": "string"}
                        }
                    }
                }
            }
        }
    }
}
validator = Draft7Validator(schema)

def validate_seed_example(example):
    """Validate a seed example against the schema"""
    try:
        validator.validate(example)
        return True, None
    except Exception as e:
        return False, str(e)

test_examples = [SEED_EXAMPLES[2], SEED_EXAMPLES[3]]

for i, example in enumerate(test_examples):
    is_valid, error = validate_seed_example(example)
    if is_valid:
        print(f"✓ Example {i+1} ({example['id']}) validates successfully")
    else:
        print(f"✗ Example {i+1} ({example['id']}) validation error: {error}")

print(f"\nSchema successfully defined based on the structure of:")
print(f"1. {SEED_EXAMPLES[2]['id']}")
print(f"2. {SEED_EXAMPLES[3]['id']}")

def validate_all_seed_examples():
    """Validate all seed examples against the schema"""
    results = []
    
    for i, example in enumerate(SEED_EXAMPLES):
        is_valid, error = validate_seed_example(example)
        results.append({
            'index': i,
            'id': example['id'],
            'valid': is_valid,
            'error': error
        })
        
        if is_valid:
            print(f"✓ Example {i} ({example['id']}) validates successfully")
        else:
            print(f"✗ Example {i} ({example['id']}) validation error:")
            print(f"  {error}")
            print()
    
    return results

# Run validation on all examples
print("Validating all seed examples against the schema:\n")
validation_results = validate_all_seed_examples()

valid_count = sum(1 for r in validation_results if r['valid'])
total_count = len(validation_results)

print(f"\n=== VALIDATION SUMMARY ===")
print(f"Valid examples: {valid_count}/{total_count}")
print(f"Invalid examples: {total_count - valid_count}")

if valid_count < total_count:
    print(f"\nInvalid examples that need fixing:")
    for r in validation_results:
        if not r['valid']:
            print(f"- {r['id']} (index {r['index']})")

✓ Example 1 (financial_trading_system) validates successfully
✓ Example 2 (healthcare_management_system) validates successfully

Schema successfully defined based on the structure of:
1. financial_trading_system
2. healthcare_management_system
Validating all seed examples against the schema:

✓ Example 0 (ecommerce_platform) validates successfully
✓ Example 1 (social_media_platform) validates successfully
✓ Example 2 (financial_trading_system) validates successfully
✓ Example 3 (healthcare_management_system) validates successfully
✓ Example 4 (educational_lms_platform) validates successfully

=== VALIDATION SUMMARY ===
Valid examples: 5/5
Invalid examples: 0


# 5. Generation Function

Below function takes a seed, calls the LLM, and returns validated examples.

In [ ]:
import json
import time
from groq import Groq

PROMPT_TEMPLATE = '''
You are a database architect expert. Given a business process description, generate a complete database schema specification according to the provided schema format.

### Instructions:
Given the following business process description, generate a JSON object that follows the exact structure shown in the examples.

Business Process:
"""
{description}
"""

### Requirements:
1. Generate a complete JSON object with keys: "id", "business_requirements", "domain_context", "schema_design", "performance_considerations", "queries"
2. The "schema_design" must include "tables" and "relationships"
3. Each table must have "name", "columns", and "indexes" arrays
4. Each column must have "name", "type", and "constraints" properties
5. Each index must have "name" and "columns" properties
6. Include realistic SQL queries in the "queries" section
7. Make the schema comprehensive and production-ready

### Example Structure Reference:
```json
{{
  "id": "example_system",
  "business_requirements": "...",
  "domain_context": "...",
  "schema_design": {{
    "tables": [
      {{
        "name": "table_name",
        "columns": [
          {{"name": "id", "type": "BIGINT", "constraints": ["PRIMARY KEY", "AUTO_INCREMENT"]}},
          {{"name": "name", "type": "VARCHAR(255)", "constraints": ["NOT NULL"]}}
        ],
        "indexes": [
          {{"name": "idx_name", "columns": ["name"]}}
        ]
      }}
    ],
    "relationships": [
      {{"from": "table1.foreign_key", "to": "table2.id", "type": "FOREIGN KEY", "on_delete": "CASCADE"}}
    ]
  }},
  "performance_considerations": {{
    "partitioning": ["..."],
    "indexing_strategy": ["..."],
    "caching": ["..."]
  }},
  "queries": {{
    "common_operations": [
      {{
        "description": "Query description",
        "sql": "SELECT ...",
        "performance_notes": "Notes about performance"
      }}
    ],
    "analytics_queries": [
      {{
        "description": "Analytics query description", 
        "sql": "SELECT ...",
        "performance_notes": "Performance considerations"
      }}
    ]
  }}
}}
```

Output only the complete JSON object. Ensure it's valid JSON and follows the schema exactly.
'''

# 6. Batch Generation

Generate ~500 examples by iterating over seeds and paraphrasing.

# 7. Save to JSONL

# 8. Upload to Hugging Face Datasets